In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("course_description.csv", encoding_errors="ignore", usecols=["Desc"])

In [3]:
content_list = df["Desc"].tolist()

In [5]:
from bs4 import BeautifulSoup as bs

string_list = [ bs(i, "html.parser").get_text() for i in content_list ] # remove html tag

In [6]:
import re # regex

http_pattern = re.compile("(\w+:\/\/\S+)") # Hyperlink
hash_pattern = re.compile("(#[A-Za-z0-9_]+)") # Hashtag 
amp_pattern = re.compile("&([0-9a-zA-Z]*)") # amp pattern
tag_pattern = re.compile("(@[A-Za-z0-9_]+)") # @민수 @Davi

emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)  # 모든 이상한 기호들 삭제

# emoji

filter_list = []

for i in string_list:

    i = re.sub(http_pattern, '', i)
    i = re.sub(hash_pattern, '', i)
    i = re.sub(amp_pattern, '', i)
    i = re.sub(tag_pattern, '', i)
    i = re.sub(emoji_pattern, '', i)
    filter_list.append(i)
    
print(filter_list)

["Introduction to programming practice using Python. Analysis and formulation of problems for computer solution. Systematic design, construction, and testing of programs. Substantial programming assignments in Python. See professor's website for an updated syllabus.", 'This is an introductory course on the fundamentals of computer programming. I see this class as an opportunity for you, the student, to see what computer programming is all about and (more importantly) to see whether you want to spend the next few years doing more of it. This course will include weekly programming projects, readings, a midterm, and final examinations. Class participation is not optional.', 'Intended for students who have completed COMP_SCI 111, but don’t have any other formal Computer Science background. It will provide an introduction to object-oriented programming in Python, preparing students for future courses such as COMP_SCI 211.\xa0', 'Overview of computer engineering design. Number systems and Bo

In [7]:
# What is "\xa0" ?
# https://stackoverflow.com/questions/10993612/how-to-remove-xa0-from-string-in-python
# \xa0 is actually non-breaking space in Latin1 (ISO 8859-1), also chr(160). You should replace it with a space.

# string = string.replace(u'\xa0', u' ')

In [12]:
filter_list = [ i.replace(u"\xa0", u"") for i in filter_list ]
print(filter_list)

["Introduction to programming practice using Python. Analysis and formulation of problems for computer solution. Systematic design, construction, and testing of programs. Substantial programming assignments in Python. See professor's website for an updated syllabus.", 'This is an introductory course on the fundamentals of computer programming. I see this class as an opportunity for you, the student, to see what computer programming is all about and (more importantly) to see whether you want to spend the next few years doing more of it. This course will include weekly programming projects, readings, a midterm, and final examinations. Class participation is not optional.', 'Intended for students who have completed COMP_SCI 111, but don’t have any other formal Computer Science background. It will provide an introduction to object-oriented programming in Python, preparing students for future courses such as COMP_SCI 211.', 'Overview of computer engineering design. Number systems and Boolea

In [13]:
from nltk import word_tokenize
from nltk.tag import pos_tag
from nltk import WordNetLemmatizer

with open("basic_stopword.txt", "r", encoding="utf-8") as f: #Encoding Type Error
    
    # euc-kr, cp949, utf-16
    
    stopword = f.read()

stopword_list = stopword.split() 
wlem = WordNetLemmatizer() # nltk -> Lemmatize ( Historical -> History ) 

In [14]:
topic_list = []

for i in filter_list:

    alive_list = []
    
    # word_tokenize -> 1개의 문장을 Tokenize -> [ "I" , "Love", "You" ]
    # [ ("I", OO"), ("Love", "XX") ..... ]
    for pos in (pos_tag(word_tokenize(i))):

        if (pos[1].startswith("JJ") or pos[1].startswith("NN") or pos[1].startswith("RB") or pos[1].startswith("VB")) \
                and ("'" not in pos[0]):

                pre_keyword = pos[0].lower().strip() # pos_tagging -> keyword ( Wayne )

                if pre_keyword in stopword_list:
                    continue
                if len(pre_keyword) < 3:
                    continue 

                keyword = wlem.lemmatize(pre_keyword) # keyword -> lemmatize -> keyword ( Wa )

                if len(keyword) < 3:
                    continue
                if keyword not in stopword_list:
                    alive_list.append(keyword)

    topic_list.append(alive_list)

print(topic_list)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/gimseyeong/nltk_data'
    - '/Users/gimseyeong/anaconda3/envs/nlp/nltk_data'
    - '/Users/gimseyeong/anaconda3/envs/nlp/share/nltk_data'
    - '/Users/gimseyeong/anaconda3/envs/nlp/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [36]:
import gensim as gs #gensim 
import pyLDAvis.gensim_models

In [37]:
gs_dict = gs.corpora.Dictionary(topic_list)
gs_corpus = [gs_dict.doc2bow(text) for text in topic_list ]

In [38]:
alpha_value = [0.01, 0.03, 0.1, 0.3, 0.5, 0.7] # 사용자가 임의로 값을 수정, 삭제할 수 있습니다.
k_start_value = 2
k_end_value = 15 # k는 토픽의 범위를 담당합니다. start ~ end
# beta 값은 1/k 로 계산합니다.
passes_value = 100
iteration_value = 1000

In [ ]:
for alpha in alpha_value:

    for i in range(k_start_value, k_end_value+1):

        print(i)
        beta_value = 1 / k_start_value
        # lda_model = gs.models.ldamodel.LdaModel(gs_corpus, num_topics=i, id2word=gs_dict, chunksize=10000, update_every=1, passes=passes_value, iterations=iteration_value, alpha=alpha, eta=beta_value, eval_every=None, per_word_topics=True)

        lda_model = gs.models.ldamulticore.LdaMulticore(gs_corpus, num_topics=i, id2word=gs_dict, chunksize=2000,
                                                passes=passes_value, iterations=iteration_value,
                                                alpha=alpha, eta=beta_value, eval_every=None, per_word_topics=True, workers=4)

        coherence_model_lda = gs.models.CoherenceModel(model=lda_model, texts=topic_list, dictionary=gs_dict, coherence='c_v', topn=10)
        coherence_lda = coherence_model_lda.get_coherence()


        lda_model.print_topics()

        lda_visual = pyLDAvis.gensim_models.prepare(lda_model, gs_corpus, gs_dict)
        pyLDAvis.save_html(lda_visual, "./gensim_data/" + "alpha" + f"_{alpha}" + f"_topic_{i}_{coherence_lda}" + ".html")
        # pyLDAvis.save_html(lda_visual, f"alpha" + f"_{alpha}" + f"_topic_{i}_{coherence_lda}" + ".html")

2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2


In [32]:
!pip install blosc2
!pip install bertopic
from bertopic import BERTopic

                                              0.0/2.3 MB ? eta -:--:--
     --                                       0.1/2.3 MB 3.6 MB/s eta 0:00:01
     ------                                   0.4/2.3 MB 4.6 MB/s eta 0:00:01
     ------------                             0.7/2.3 MB 5.7 MB/s eta 0:00:01
     --------------------                     1.2/2.3 MB 6.7 MB/s eta 0:00:01
     ----------------------------             1.6/2.3 MB 7.3 MB/s eta 0:00:01
     -----------------------------------      2.0/2.3 MB 7.6 MB/s eta 0:00:01
     ---------------------------------------- 2.3/2.3 MB 7.6 MB/s eta 0:00:00
                                              0.0/85.7 kB ? eta -:--:--
     ---------------------------------------- 85.7/85.7 kB ? eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, but you have blosc2 2.2.6 which is incompatible.


In [33]:
topic_model = BERTopic()

In [47]:
corpus = " ".join(filter_list)
topics, probs = topic_model.fit_transform(filter_list)

In [49]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4,-1_soon_coming_under_construction,"[soon, coming, under, construction, announced,...","[Under Construction - Coming Soon, Under Const..."
1,0,125,0_and_the_of_to,"[and, the, of, to, in, will, course, this, for...",[This course aims to cover various tools in th...
2,1,13,1_this_course_be_may,"[this, course, be, may, is, or, not, the, page...","[This course is not currently offered., *Pleas..."


In [52]:
topic_model.get_topics(5)

{'Main': {-1: [('soon', 1.408922892339867),
   ('coming', 1.408922892339867),
   ('under', 1.3514446089355179),
   ('construction', 1.2396957432984617),
   ('announced', 0.5428430123961864),
   ('be', 0.2767264487818014),
   ('to', 0.1661425667542997),
   ('', 1e-05),
   ('', 1e-05),
   ('', 1e-05)],
  0: [('and', 0.10756446970469132),
   ('the', 0.09391860801258167),
   ('of', 0.07798637431977755),
   ('to', 0.07555688299213927),
   ('in', 0.0624838489143789),
   ('will', 0.048946867950561426),
   ('course', 0.04408845468100664),
   ('this', 0.0403752648390842),
   ('for', 0.03898756912773047),
   ('is', 0.03648565958301419)],
  1: [('this', 0.15139356218125027),
   ('course', 0.14642342121849322),
   ('be', 0.13952594056225281),
   ('may', 0.11725168387739221),
   ('is', 0.1153053437226491),
   ('or', 0.1119489234002501),
   ('not', 0.10625404108358935),
   ('the', 0.09084295104191795),
   ('page', 0.08879766128192439),
   ('currently', 0.08879766128192439)]}}